### Imports inicales

In [3]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from IPython.display import display

#Libreria para plotear
# %pip install --upgrade plotly
import plotly.express as px

# Librerias utiles
# %pip install --upgrade scikit-learn
from sklearn.preprocessing import StandardScaler

In [5]:
def eda(dataframe):
    pd.options.display.max_rows = 200
    pd.set_option('display.max_columns', None)

    print('1.- El DataFrame tiene', dataframe.shape[0],'filas y', dataframe.shape[1], 'columnas\n')
    print('2.- El DataFrame está compuesto por las siguientes columnas: ', list(dataframe.columns),'\n')
    print('3.- Ejemplos de filas del DataFrame:\n')
    print('Primeras 5 filas:')
    display(dataframe.head(5))
    print('Últimas 5 filas:')
    display(dataframe.tail(5))
    print('Muestreo aleatorio de 5 filas:')
    display(dataframe.sample(5))
    print('4.- Descripción numérica del Dataframe:\n')
    display(dataframe.describe())
    print('5.- Cantidad de valores nulos por columna:')
    display(dataframe.isna().sum())
    print('6.- Cantidad de valores únicos por columna:')
    unique_values = dataframe.nunique()
    display(unique_values)
    print('7.- Filas totales duplicadas en el DataFrame:')
    display(dataframe.duplicated().sum())
    print('8.- Filas duplicadas en el DataFrame:')
    display(dataframe[dataframe.duplicated(keep=False)])

In [7]:
restaurantes = pd.read_csv("./restaurants.csv")
menus = pd.read_csv("./restaurant-menus.csv")

### EDA

Restaurantes

In [8]:
eda(restaurantes)

1.- El DataFrame tiene 63469 filas y 11 columnas

2.- El DataFrame está compuesto por las siguientes columnas:  ['id', 'position', 'name', 'score', 'ratings', 'category', 'price_range', 'full_address', 'zip_code', 'lat', 'lng'] 

3.- Ejemplos de filas del DataFrame:

Primeras 5 filas:


,id,position,name,score,ratings,category,price_range,full_address,zip_code,lat,lng
0,1,19,PJ Fresh (224 Daniel Payne Drive),NaN,NaN,"Burgers, American, Sandwiches",$,"224 Daniel Payne Drive, Birmingham, AL, 35207",35207,33.562365,-86.830703
1,2,9,J' ti`'z Smoothie-N-Coffee Bar,NaN,NaN,"Coffee and Tea, Breakfast and Brunch, Bubble Tea",NaN,"1521 Pinson Valley Parkway, Birmingham, AL, 35217",35217,33.583640,-86.773330
2,3,6,Philly Fresh Cheesesteaks (541-B Graymont Ave),NaN,NaN,"American, Cheesesteak, Sandwiches, Alcohol",$,"541-B Graymont Ave, Birmingham, AL, 35204",35204,33.509800,-86.854640
3,4,17,Papa Murphy's (1580 Montgomery Highway),NaN,NaN,Pizza,$,"1580 Montgomery Highway, Hoover, AL, 35226",35226,33.404439,-86.806614
4,5,162,Nelson Brothers Cafe (17th St N),4.7,22.0,"Breakfast and Brunch, Burgers, Sandwiches",NaN,"314 17th St N, Birmingham, AL, 35203",35203,33.514730,-86.811700


Últimas 5 filas:


,id,position,name,score,ratings,category,price_range,full_address,zip_code,lat,lng
63464,63465,2,IHOP (4321 N 10th St),4.6,56.0,"American, Breakfast and Brunch, burger, Comfor...",$,"4321 N 10th St, MC ALLEN, TX, 78504",78504,26.242853,-98.223984
63465,63466,29,Feldman's - N 10th Street,NaN,NaN,NaN,NaN,"3010 N 10th Street, Mc Allen, TX,",NaN,26.230764,-98.224922
63466,63467,27,Feldman's - N 23rd St,NaN,NaN,NaN,NaN,"6101 N 23rd Street #B, Mc Allen, TX,",NaN,26.260353,-98.236755
63467,63468,19,Church's Chicken (620 S. 10th Street),4.3,17.0,"Fast Food, Family Meals",$,"620 S. 10th Street, McAllen, TX, 78501",78501,26.197333,-98.231369
63468,63469,9,7-Eleven (1600 S 23rd St),NaN,NaN,"Everyday Essentials, Convenience, Home &amp; P...",$,"1600 S 23rd St, Mcallen, TX, 78503",78503,26.191710,-98.247880


Muestreo aleatorio de 5 filas:


...

id              63469
position          300
name            60865
score              33
ratings           433
category        15574
price_range         5
full_address    54946
zip_code         3082
lat             57227
lng             57249
dtype: int64

7.- Filas totales duplicadas en el DataFrame:


np.int64(0)

8.- Filas duplicadas en el DataFrame:


,id,position,name,score,ratings,category,price_range,full_address,zip_code,lat,lng


---
Menus

In [9]:
eda(menus)

1.- El DataFrame tiene 5117217 filas y 5 columnas

2.- El DataFrame está compuesto por las siguientes columnas:  ['restaurant_id', 'category', 'name', 'description', 'price'] 

3.- Ejemplos de filas del DataFrame:

Primeras 5 filas:


,restaurant_id,category,name,description,price
0,1,Extra Large Pizza,Extra Large Meat Lovers,Whole pie.,15.99 USD
1,1,Extra Large Pizza,Extra Large Supreme,Whole pie.,15.99 USD
2,1,Extra Large Pizza,Extra Large Pepperoni,Whole pie.,14.99 USD
3,1,Extra Large Pizza,Extra Large BBQ Chicken &amp; Bacon,Whole Pie,15.99 USD
4,1,Extra Large Pizza,Extra Large 5 Cheese,Whole pie.,14.99 USD


Últimas 5 filas:


,restaurant_id,category,name,description,price
5117212,63469,Other Essentials,Composition Notebook,NaN,4.38 USD
5117213,63469,Other Essentials,Fancy Fest Savory Salmon - 3oz,NaN,1.19 USD
5117214,63469,Other Essentials,Bicycle Playing Cards,NaN,3.83 USD
5117215,63469,Other Essentials,Tidy Cat Liter - 10lbs,NaN,4.38 USD
5117216,63469,Other Essentials,7-Select Heavy Duty Foam Cooler,NaN,6.8 USD


Muestreo aleatorio de 5 filas:


...

restaurant_id     63309
category          73048
name             948402
description      920266
price              7138
dtype: int64

7.- Filas totales duplicadas en el DataFrame:


np.int64(34918)

8.- Filas duplicadas en el DataFrame:


,restaurant_id,category,name,description,price
379,7,Nigiri / Sashimi,Tuna Tataki,Seared tuna.,6.0 USD
380,7,Nigiri / Sashimi,Amaebi,Sweet shrimp.,7.0 USD
381,7,Nigiri / Sashimi,Amaebi,Sweet shrimp.,7.0 USD
388,7,Nigiri / Sashimi,Tuna Tataki,Seared tuna.,6.0 USD
5838,80,Naan,Tandoori Paratha,NaN,4.79 USD
...,...,...,...,...,...
5117202,63469,Other Essentials,7-Select Heavy Duty Foam Cooler,NaN,6.8 USD
5117205,63469,Other Essentials,Purina Dog Chow - 4.4lbs,NaN,7.68 USD
5117207,63469,Other Essentials,7-Select Flexible Utility Lighter,NaN,5.48 USD
5117210,63469,Other Essentials,Spiral Notebook,NaN,5.48 USD


In [15]:
# Diccionario de rangos de ZIP codes por estado
zip_code_ranges = {
    'AL': (35000, 36999), 'AK': (99500, 99999), 'AZ': (85000, 86599), 'AR': (71600, 72999), 'CA': (90000, 96199),
    'CO': (80000, 81999), 'CT': (6000, 6999), 'DE': (19700, 19999), 'FL': (32000, 34999), 'GA': (30000, 31999),
    'HI': (96700, 96999), 'ID': (83200, 83899), 'IL': (60000, 62999), 'IN': (46000, 47999), 'IA': (50000, 52999),
    'KS': (66000, 67999), 'KY': (40000, 42999), 'LA': (70000, 71599), 'ME': (3900, 4999), 'MD': (20600, 21999),
    'MA': (1000, 2799), 'MI': (48000, 49999), 'MN': (55000, 56799), 'MS': (38600, 39999), 'MO': (63000, 65999),
    'MT': (59000, 59999), 'NE': (68000, 69399), 'NV': (88900, 89999), 'NH': (3000, 3899), 'NJ': (7000, 8999),
    'NM': (87000, 88499), 'NY': (10000, 14999), 'NC': (27000, 28999), 'ND': (58000, 58899), 'OH': (43000, 45999),
    'OK': (73000, 74999), 'OR': (97000, 97999), 'PA': (15000, 19699), 'RI': (2800, 2999), 'SC': (29000, 29999),
    'SD': (57000, 57999), 'TN': (37000, 38599), 'TX': (75000, 79999), 'UT': (84000, 84999), 'VT': (5000, 5999),
    'VA': (20100, 24699), 'WA': (98000, 99499), 'WV': (24700, 26999), 'WI': (53000, 54999), 'WY': (82000, 83199)
}

# Función para obtener el estado desde un ZIP code
def get_state_from_zip(zip_code):
    try:
        zip_code = int(zip_code)
        for state, (low, high) in zip_code_ranges.items():
            if low <= zip_code <= high:
                return state
        return None
    except ValueError:
        return None


In [16]:
import pandas as pd

# Cargar los datos
data = restaurantes

# Limpiar los códigos postales y convertirlos a enteros
data['zip_code'] = pd.to_numeric(data['zip_code'], errors='coerce')

# Aplicar la función para obtener el estado desde el código postal
data['state'] = data['zip_code'].apply(get_state_from_zip)

# Eliminar filas sin estado (valores fuera de EE.UU. o ZIPs inválidos)
data_us = data.dropna(subset=['state'])

# Explota las categorías en filas para poder contarlas
categories_expanded = data_us.dropna(subset=['category']).copy()
categories_expanded['category'] = categories_expanded['category'].str.split(', ')
categories_expanded = categories_expanded.explode('category')

# Contar las categorías por estado
category_counts = categories_expanded.groupby(['state', 'category']).size().reset_index(name='count')

# Para cada estado, obtener las 3 categorías más comunes
top_categories_per_state = category_counts.groupby('state').apply(lambda x: x.nlargest(3, 'count')).reset_index(drop=True)

# Guardar el resultado en un archivo CSV para usarlo con D3.js
top_categories_per_state.to_csv('top_categories_per_state_by_zip.csv', index=False)

print("Proceso completado. El archivo 'top_categories_per_state_by_zip.csv' contiene las tres categorías más comunes por estado basado en ZIP code.")


Proceso completado. El archivo 'top_categories_per_state_by_zip.csv' contiene las tres categorías más comunes por estado basado en ZIP code.


/tmp/ipykernel_14902/2777172149.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_categories_per_state = category_counts.groupby('state').apply(lambda x: x.nlargest(3, 'count')).reset_index(drop=True)
